In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn import preprocessing, model_selection
import random
import scipy.io as io
import collections

In [ ]:
data = io.loadmat('/content/WLDataCW (3).mat')
data_only = data["data"]
data_label = data["label"]
print(data_only.shape)
print(type(data_only))
print(data_label.shape)
#62*512*360, where 62 is the number of channels, 512 is the data points for each sample,
#and 360 is the number of samples.
label_all = np.reshape(data_label, -1) 
print(label_all.shape) 
data_all = np.moveaxis(data_only, 2, 0)
#The third dimension (index 2) of the data only array is moved to the top position using this line's use of NumPy's moveaxis function (index 0). As a result, the array's original (height, width, channels) shape is effectively changed to (channels, height, width).
#The variable data all is given the resulting array.

print(data_all.shape)

#This line resizes the array data label using the NumPy module. The second option, -1, instructs NumPy to use the original array's form to determine the size of the reconfigured array automatically.
 #The variable label all is given ownership of the reshaped array that results.


In [ ]:
num_classes = len(np.unique(data_label))
print(num_classes)

counter = collections.Counter(np.reshape(data_label, -1))
print(counter)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    data_all, label_all, test_size=0.2, random_state=42, shuffle=True
)

print(
    f"Length of x_train : {len(x_train)}\nLength of x_test : {len(x_test)}\nLength of y_train : {len(y_train)}\nLength of y_test : {len(y_test)}"
)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

train_images0 = x_train


In [ ]:
if len(x_train.shape) == 3:  # images must be rank-4 tensors [number_of_images, dimension1, dimension2, number_of_channels]
    # add a single channel to these black-and-white images
    x_train = x_train.reshape(list(x_train.shape)+[1])
    x_test = x_test.reshape(list(x_test.shape)+[1])
    print("Reshaped images from ", train_images0.shape, "to", x_train.shape, "so that 'channel' dimension exists")

num_classification_categories = num_classes

In [ ]:
# build model with CNN architecture
model = keras.Sequential()

# YOUR CODE HERE (do NOT specify the input shape in the first layer)
model.add(layers.Conv2D(16, (3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classification_categories, activation='softmax'))

In [ ]:
model.build(input_shape=(None,) + x_train.shape[1:])
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam()

model.compile(optimizer=optimizer,  
              loss=keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                batch_size=128,
                epochs=50,
                validation_data=(x_test, y_test))

In [ ]:
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')